1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸린다.
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200 # 청크를 겹치게 한다 (앞뒤 문맥을 같이 줌으로써 확률이올라감)
)
loader = Docx2txtLoader('./tax_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list[62]

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')


In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name="tax-markdown"

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [ ]:
retriever = database.as_retriever(search_kwargs={'k': 5})
retriever.invoke(query)


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt") # QA 프롬프트를 제공한다

In [ ]:
from langchain.chains import RetrievalQA

retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
message = qa_chain({"query": query})
message["result"]